# Driver Review Notebook

Generate reports for a particular driver.

In [14]:
!pip3 install --upgrade --force-reinstall .

Processing /Users/ajh59/Dropbox/WRC/wrcX
  Found existing installation: wrcX 0.1
    Uninstalling wrcX-0.1:
      Successfully uninstalled wrcX-0.1
  Running setup.py install for wrcX ... - \ done


In [1]:
rally="Portugal"
wrcwebnum=226
wrcstages=17
year=2017
year,rallyid,rally,maxstages=(year,wrcwebnum,rally,wrcstages) 

rallydet=(year,wrcwebnum,rally)
YEAR,THISRALLY=(year,wrcwebnum)
first_stage=1
stages=list(range(first_stage,maxstages+1))

In [3]:
import wrcX
wrcX.init({'year':year,'rallyid':rallyid},forceReload=True)

('Used cache for http://www.wrc.com/live-ticker/daten/2017/226/entry.226.html:', False)
('Used cache for http://www.wrc.com/live-ticker/daten/2017/226/itinerary.226.all.html:', False)
('Used cache for http://www.wrc.com/live-ticker/daten/2017/226/retirements.226.all.html:', False)
('Used cache for http://www.wrc.com/live-ticker/daten/2017/226/penalties.226.all.html:', False)
('Used cache for http://www.wrc.com/live-ticker/daten/2017/226/stage.226.1.overall.all.html:', False)
('Used cache for http://www.wrc.com/live-ticker/daten/2017/226/stage.226.2.overall.all.html:', False)
('Used cache for http://www.wrc.com/live-ticker/daten/2017/226/stage.226.3.overall.all.html:', False)
('Used cache for http://www.wrc.com/live-ticker/daten/2017/226/stage.226.4.overall.all.html:', False)
('Used cache for http://www.wrc.com/live-ticker/daten/2017/226/stage.226.5.overall.all.html:', False)
('Used cache for http://www.wrc.com/live-ticker/daten/2017/226/stage.226.6.overall.all.html:', False)
('Used cac

In [4]:
import nbformat as nb
import nbformat.v4.nbbase as nb4

import os

import wrcX.core.data as d
from wrcX.reports.utils import _limitToGroupClass, driverFromCar
from wrcX.reports.overallSlides import addSlideComponent, add_fig

In [ ]:
def carReport(rallydet,carNum,groupClass='RC1'):
    imgpath='images/test{}_{}'.format(rallydet[0],rallydet[1])
    
    driver=driverFromCar(d.df_entry,carNum)
    if not os.path.exists(imgpath):
        os.makedirs(imgpath)
    
    test=nb4.new_notebook()
    
    addSlideComponent(test,'# WRC Rally - {}, {}'.format(rallydet[2],rallydet[0]),'slide')
    addSlideComponent(test,'## Driver Analysis - {} ({})'.format(driver,carNum))
    
    for key,day in d.df_stagedesc[d.df_stagedesc['stage'].str.startswith('SS')].groupby('day'):
        for key2,section in day.groupby('section'):
            txt='# Day {} ({})\n'.format(key,'/'.join(day['date'].dt.strftime('%d/%m/%Y').unique())) + \
                '## Section {} ({})'.format(key2,'/'.join(section['partofday'].unique()))
            addSlideComponent(test,txt,'slide')

            #Within each section are stage rows
            for key,row in section.iterrows():
                if d.df_stagedesc.ix[key]['stagenum']> wrcX.core.stagesuptoandincluding : break
                
                ddf_sectors=d.df_splitSectorTimes_all[row['stagenum']-int(wrcX.core.first_stage)]
                df_splitTimes=d.df_splitTimes_all[row['stagenum']-int(wrcX.core.first_stage)]
                
                addSlideComponent(test,'## {} - stage sector delta time (seconds) compared to {} stage sector time'.format(row['stage'],driver),'slide')
                fig=wrcX.chart_stage_sector_delta(year,rallyid,d.df_entry,df_splitTimes,ddf_sectors,
                                                  rebase=carNum,ylim=[-50,50],keyHighlight=str(carNum))
                txt=add_fig(fig,'chart_stage_sector_delta{}_{}'.format(carNum,row['stage']))
                addSlideComponent(test,txt)
                txt1='*This chart allows you to compare the __on stage__ performance of each driver. It displays the "sector time" of each driver (the time between each split point along the stage), rebased relative to car {}.*'.format(carNum)
                addSlideComponent(test,txt1)
                
                addSlideComponent(test,'## {} - stage sector time (seconds)'.format(row['stage']),'subslide')
                xcols=[c for c in ddf_sectors.columns if c.startswith('sector')]
                if len(xcols)==1: xcols=[]
                tcols=['start','carNo','driverName']+xcols+['time_stageTime']
                pretxt=ddf_sectors.sort_values('time_stageTime')[tcols][:10]
                pretxt['start']=pretxt['start'].astype(int)
                txt=pretxt.set_index('start').to_html()
                addSlideComponent(test,txt)
                
                addSlideComponent(test,'## {} - stage split delta time (seconds) compared to {}'.format(row['stage'],driver),'subslide')
                fig=wrcX.chart_stage_delta_s(year,rallyid,d.df_entry,df_splitTimes,
                                             rebase=carNum,ylim=[-50,50],keyHighlight=driver)
                txt=add_fig(fig,'split_delta_driver{}_{}'.format(carNum,row['stage']))
                addSlideComponent(test,txt)
                txt1='*This chart allows you to compare the __on stage__ performance of each driver, with split times rebased relative to car {} on stage.*'.format(carNum)
                #txt1=txt1+'\n*The leader within the stage at each split point is highlighted.*'
                addSlideComponent(test,txt1)

                
            
                nbf='wrcTest-slides-car-{}-{}.ipynb'.format(carNum,rallydet[2])
                nb.write(test,nbf)
                
carNum=8#4
carReport(rallydet,carNum)

In [ ]:
nbf='wrcTest-slides-car-{}-{}.ipynb'.format(carNum,rallydet[2])
!jupyter nbconvert $nbf --to slides --post serve